In [ ]:
import os
from argparse import ArgumentParser
from mmdet.apis import show_result_pyplot
from mmfewshot.detection.apis import (
    inference_detector,
    init_detector,
    process_support_images,
)

In [ ]:
import numpy as np
from easydict import EasyDict as edict
from PIL import Image

In [ ]:
idx = 1
image_path = f"/home/jongphago/mmfewshot/data/fsod/mf_210223/images/x{idx:03d}.JPG"
size = Image.open(image_path).size

args = edict(
    {
        "support_images_dir": "/home/jongphago/mmfewshot/demo/demo_detection_images/support_images_2",
        # "image": "/home/jongphago/mmfewshot/demo/demo_detection_images/query_images/query_manhole.jpg",
        "image": image_path,
        "label": f"/home/jongphago/mmfewshot/data/fsod/mf_210223/labels/x{idx:03d}.txt",
        "size":size,
        "config": "/home/jongphago/mmfewshot/configs/detection/attention_rpn/coco/attention-rpn_r50_c4_4xb2_coco_base-training.py",
        "checkpoint": "/home/jongphago/mmfewshot/work_dirs/attention-rpn_r50_c4_4xb2_coco_base-training_20211102_003348-da28cdfd.pth",
        "score_thr": 0.2,
        "device": "cuda:0",
    }
)

In [ ]:
def main(args):
    # build the model from a config file and a checkpoint file
    model = init_detector(args.config, args.checkpoint, device=args.device)
    # prepare support images, each demo image only contain one instance
    files = os.listdir(args.support_images_dir)
    support_images = [os.path.join(args.support_images_dir, file) for file in files]
    classes = [file.split(".")[0] for file in files]
    support_labels = [[file.split(".")[0]] for file in files]
    process_support_images(model, support_images, support_labels, classes=classes)
    # test a single image
    result = inference_detector(model, args.image)
    # show the results
    show_result_pyplot(model, args.image, result, score_thr=args.score_thr)
    return result

In [ ]:
result = main(args)
result

In [ ]:
# %pip install scikit-learn

In [ ]:
import torch
from torchvision.ops import box_iou
from sklearn.metrics import average_precision_score

In [ ]:
preds = [[idx, v[-1], *v[:-1]] for idx, values in enumerate(result) for v in values]
preds = np.array(preds, dtype=int)[:, 2:]

In [ ]:
with open(args.label, 'r') as f:
    lines = f.readlines()
    lines = [line.strip().split() for line in lines]
    lines = [[int(line[0]), float(line[1]), float(line[2]), float(line[3]), float(line[4])] for line in lines]
    ground_truth = np.array(lines)
ground_truth = np.multiply(ground_truth, [1] + args.size * 2).astype(int)[:, 1:]

In [ ]:
def convert_center_to_corner(center_x, center_y, width, height):
    left_x = center_x - (width / 2)
    top_y = center_y - (height / 2)
    right_x = center_x + (width / 2)
    bottom_y = center_y + (height / 2)

    return [left_x, top_y, right_x, bottom_y]

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
from matplotlib.pyplot import plt

# 이미지 파일 경로 설정
image_file_path = args.image

# 넘파이 배열로 좌표 설정
red_coordinates = np.array([[155, 100, 186, 216], [194, 400, 358, 450]])

blue_coordinates = np.array([[278, 424, 169, 42]])

# 이미지 파일 읽기
image = Image.open(image_file_path)

# 빨간색 박스를 그리기 위한 ImageDraw 객체 생성
draw_red = ImageDraw.Draw(image)

# 빨간색 박스 그리기
for coord in red_coordinates:
    x1, y1, x2, y2 = coord
    draw_red.rectangle([x1, y1, x2, y2], outline="red", width=2)

# 파란색 박스를 그리기 위한 ImageDraw 객체 생성
draw_blue = ImageDraw.Draw(image)

# 파란색 박스 그리기
for coord in blue_coordinates:
    x1, y1, x2, y2 = convert_center_to_corner(*coord)
    draw_blue.rectangle([x1, y1, x2, y2], outline="blue", width=2)

# 변경된 이미지를 저장하거나 표시
image.save("output_image_with_boxes.jpg")
plt.imshow(image)
plt.show()